In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# 기본 웹드라이버로 Chrome 실행
driver = webdriver.Chrome()

# URL 접속
url = 'https://www.volvocar.co.kr/buy/purchase/find-a-showroom/?type=service'
driver.get(url)

# 페이지 로드 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.result_list li')))

# 데이터 수집
service_centers = []

# 모든 li 요소 가져오기
list_items = driver.find_elements(By.CSS_SELECTOR, 'ul.result_list li')

for item in list_items:
    # 업체명 추출
    company_name = item.find_element(By.CSS_SELECTOR, 'div.btn .ti').text.strip()
    
    # 주소, 전화번호, 팩스번호, 영업시간 추출 (없으면 None으로 저장)
    address = item.find_element(By.CSS_SELECTOR, 'div.addr .txt').get_attribute('innerHTML').strip() if item.find_elements(By.CSS_SELECTOR, 'div.addr .txt') else None
    phone = item.find_element(By.CSS_SELECTOR, 'div.phone .txt').get_attribute('innerHTML').strip() if item.find_elements(By.CSS_SELECTOR, 'div.phone .txt') else None
    fax = item.find_element(By.CSS_SELECTOR, 'div.fax .txt').get_attribute('innerHTML').strip() if item.find_elements(By.CSS_SELECTOR, 'div.fax .txt') else None
    opening_hours = item.find_element(By.CSS_SELECTOR, 'div.time .txt').get_attribute('innerHTML').replace('<br>', ' ').strip() if item.find_elements(By.CSS_SELECTOR, 'div.time .txt') else None
    
    # 데이터 저장
    service_centers.append({
        '업체명': company_name,
        '주소': address,
        '전화번호': phone,
        '팩스번호': fax,
        '영업시간': opening_hours
    })

# 데이터프레임 생성
df = pd.DataFrame(service_centers)

# 데이터프레임 출력
print(df)



                 업체명                                                 주소  \
0    볼보 강남 대치 서비스 센터         서울시 강남구 영동대로 313 <span>(대치동 983-13)</span>   
1    볼보 강남 삼성 서비스 센터      서울특별시 강남구 삼성로 71길 6 <span>(대치동 912-25)</span>   
2    볼보 강남 율현 서비스 센터      서울특별시 강남구 헌릉로757길 61 <span>(율현동 100-3)</span>   
3        볼보 강릉 서비스센터           강원도 강릉시 율곡로 3008 <span>(교동 713-4)</span>   
4       볼보 광안 서비스 센터        부산광역시 수영구 광남로 147 <span>(광안동 159-19)</span>   
5       볼보 광주 서비스 센터       광주광역시 서구 상무대로 1096 <span>(농성동 649-16)</span>   
6        볼보 구리 서비스센터      경기도 구리시 동구릉로 23 <span>(구리시 인창동 562-35)</span>   
7       볼보 군산 서비스 센터                                전북특별자치도 군산시 번영로 155   
8       볼보 김해 서비스 센터       경상남도 김해시 김해대로 2427 <span>(부원동 626-21)</span>   
9       볼보 대구 서비스 센터       대구광역시 동구 안심로 55길 15 <span>(동호동 108-4)</span>   
10      볼보 대전 서비스 센터         대전광역시 유성구 한밭대로 378 <span>(궁동 448-1)</span>   
11     볼보 동대문 서비스 센터       서울특별시 동대문구 천호대로 373 <span>(장안동 414-6)</span>   
12      볼보 부천 서비스 센터     

In [39]:
df['주소'] = df['주소'].str.replace('특별시|광역시|특별자치도|특별자치시|<span>', '', regex=True)
df['주소'] = df['주소'].str.replace('서울시', '서울', regex=False)
df['주소'] = df['주소'].str.replace('경기도', '경기', regex=False)
df['주소'] = df['주소'].str.replace('강원도', '강원', regex=False)
df['주소'] = df['주소'].str.replace('충청북도', '충북', regex=False)
df['주소'] = df['주소'].str.replace('충청남도', '충남', regex=False)
df['주소'] = df['주소'].str.replace('전라북도', '전북', regex=False)
df['주소'] = df['주소'].str.replace('전라남도', '전남', regex=False)
df['주소'] = df['주소'].str.replace('경상북도', '경북', regex=False)
df['주소'] = df['주소'].str.replace('경상남도', '경남', regex=False)

# '시도' 열에 주소 첫 두 글자 추가
df['시도'] = df['주소'].str[:2]

# '시군구' 열에 첫 번째와 두 번째 띄어쓰기 사이의 텍스트 추출
df['시군구'] = df['주소'].str.extract(r'^\S+\s(\S+)')[0]

df['시도+시군구'] = df['시도'] + df['시군구']

In [43]:
df.to_excel('볼보_최종.xlsx')